# <b> <u> <font color = green> CA 683 Project : Used Car Sales </u> </b>

Dataset with used Car Sales from United States

Our Dataset was downloaded from Kaggle:

https://www.kaggle.com/tsaustin/us-used-car-sales-data

Team:
- Emma Dunne
- Tony Delaney
- Meenu Matthew
- Darren Spain


## <font color = green> Import Necesscary files and read in the Used Car Data Set and other Data Sets.

In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV


In [2]:
pip install ipython[all]

     |████████████████████████████████| 204kB 19.2MB/s 
     |████████████████████████████████| 163kB 49.0MB/s 


In [4]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
!ls

/content
drive  sample_data


In [5]:
os.chdir('/content/drive/My Drive/CA683Project')
!pwd
!ls

/content/drive/My Drive/CA683Project
DisposableIncomebyState.csv   DisposableIncomeGasPriceByState.csv
DisposableIncomebyState.xlsx  used_car_sales.csv


In [6]:
df = pd.read_csv('used_car_sales.csv',sep=',')


In [8]:
df.head()

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType
0,137178,7500,2020,786**,84430,Ford,Mustang,1988,LX,5.0L Gas V8,Sedan,0,RWD
1,96705,15000,2019,81006,0,Replica/Kit Makes,Jaguar Beck Lister,1958,NaN,383 Fuel injected,Convertible,8,RWD
2,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD
3,80773,11600,2019,07852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD
4,64287,44000,2019,07728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD


### Read in Disposable income per state

create this Dataframe to capture the yearly disposable income per state and Gas price per state. This will be combined with our car sales dataframe at a later stage.

In [11]:
df_Disposable_income_Gas_price = pd.read_csv('DisposableIncomeGasPriceByState.csv',usecols=['State','Disp_Inc_2015','Disp_Inc_2016','Disp_Inc_2017','Disp_Inc_2018','Disp_Inc_2019','Disp_Inc_2020','Gas_Price_2016','Gas_Price_2017','Gas_Price_2018','Gas_Price_2019','Gas_Price_2020'])

In [ ]:
df_Disposable_income_Gas_price.head(100)

### Zip Code Manipulation

ZIP Codes are numbered with the first digit representing a certain group of U.S. states, the second and third digits together representing a region in that group (or perhaps a large city) and the fourth and fifth digits representing a group of delivery addresses within that region.
Therefore for the purpose of this exercise, we can take the first 3 digits of the zipcode as a meaningful represenatation of an area.

In [ ]:
# firstly ensure that any empty strings are replaced with NAN in zipcode coloumn
df['zipcode'].replace('', np.nan, inplace = True)
zipcode_nan = len(df['zipcode']) - df['zipcode'].count()
print ( 'There are ', zipcode_nan , ' entries with Not A Number (NAN) entries in the zipcode coloumn')

# drop the null values in zipcode coloumn
df.dropna(subset=['zipcode'], inplace= True)

# strip of last characters of the zipcode and keep the first 3 characters
df['cleaned_zipcode'] = df.zipcode.apply(lambda x: x[:3])

# ensure that only numercic values are in Zipcode and replace all others with NAN
# Note : this code removes the leading zeros from the zipcodes.
df['cleaned_zipcode'] = pd.to_numeric(df.cleaned_zipcode, errors ='coerce', downcast= 'signed')
df.dropna(subset=['cleaned_zipcode'], inplace= True)


In [14]:
# print(df.cleaned_zipcode)
print("The count of unique zipcodes in this Dataframe is ",df.cleaned_zipcode.nunique())

The count of unique zipcodes in this Dataframe is  897


Run a sanity check to ensure that the cleaned zipcode has the first 3 digits of the old zipcode

In [ ]:
df.head(16)

### Age of Car Information to be added

In [16]:
def find_age_car (row):
  age_car = row['yearsold'] - row ['Year']
  return age_car

run the function to find age of car across entire dataframe and add a new column for Age of car at time of sale.

In [17]:
df['CarAgeTimeSale'] = df.apply (find_age_car, axis=1)

In [ ]:
df.head(20)

### Break Zipcodes down to State level

the original dataset for car sales came with ZipCodes. We propose to break this down by state to allow other state level data to be considered. i.e. disposable income at a state by state level

In [55]:
def zipcode_to_state(row):
  zipcode = row['cleaned_zipcode']
  if zipcode in range(350,370):
    return 'Alabamba'
  elif zipcode in range(995,1000):
    return 'Alaska'
  elif zipcode in range(850,866):
    return 'Arizona'
  elif zipcode in range(716,730):
    return 'Arkansas'
  elif zipcode in range(900,962):
    return 'California'
  elif zipcode in range(800,817):
    return 'Colorado'
  elif zipcode in range(60,70):
    return 'Connecticut'
  elif zipcode in range(197,200):
    return 'Delaware'
  elif zipcode in range(320,350):
    return 'Florida'
  elif zipcode in range(300,320):
    return 'Georgia'
  elif zipcode in range(398,400):
    return 'Georgia'
  elif zipcode in range(967,969):
    return 'Hawaii'
  elif zipcode in range(832,839):
    return 'Idaho'
  elif zipcode in range(600,630):
    return 'Illinois'
  elif zipcode in range(460,480):
    return 'Indiana'
  elif zipcode in range(500,529):
    return 'Iowa'
  elif zipcode in range(660,680):
    return 'Kansas'
  elif zipcode in range(400,428):
    return 'Kentucky'
  elif zipcode in range(700,715):
    return 'Louisiana'
  elif zipcode in range(39,50):
    return 'Maine'
  elif zipcode in range(205,220):
    return 'Maryland'
  elif zipcode in range(10,56):
    return 'Massachusetts'
  elif zipcode in range(480,500):
    return 'Michigan'
  elif zipcode in range(550,568):
    return 'Minnesota'
  elif zipcode in range(386,398):
    return 'Mississippi'
  elif zipcode in range(630,659):
    return 'Missouri'
  elif zipcode in range(590,600):
    return 'Montana'
  elif zipcode in range(680,694):
    return 'Nebraska'
  elif zipcode in range(889,899):
    return 'Nevada'
  elif zipcode in range(30,39):
    return 'New Hampshire'
  elif zipcode in range(70,90):
    return 'New Jersey'
  elif zipcode in range(870,885):
    return 'New Mexico'  
  elif zipcode in range(5,6):
    return 'New York' 
  elif zipcode in range(100,150):
    return 'New York'
  elif zipcode in range(270,290):
    return 'North Carolina'
  elif zipcode in range(580,589):
    return 'North Dakota'
  elif zipcode in range(430,460):
    return 'Ohio'
  elif zipcode in range(730,750):
    return 'Oklahoma'
  elif zipcode in range(970,980):
    return 'Oregon'
  elif zipcode in range(150,197):
    return 'Pennsylvania'
  elif zipcode in range(6,10):
    return 'Puerto Rico'
  elif zipcode in range(28,30):
    return 'Rhode Island'  
  elif zipcode in range(290,300):
    return 'South Carolina' 
  elif zipcode in range(570,578):
    return 'South Dakota'
  elif zipcode in range(370,386):
    return 'Tennessee'
  elif zipcode in range(733,800):
    return 'Texas'
  elif zipcode in range(885,886):
    return 'Texas'
  elif zipcode in range(840,848):
    return 'Utah'
  elif zipcode in range(50,60):
    return 'Vermont'
  elif zipcode in range(889,899):
    return 'Virginia'
  elif zipcode in range(220,247):
    return 'Virginia'
  elif zipcode in range(980,995):
    return 'Washington'
  elif zipcode in range(200,206):
    return 'Washington DC'
  elif zipcode in range(247,269):
    return 'West Virginia'  
  elif zipcode in range(530,550):
    return 'Wisconsin'
  elif zipcode in range(820,835):
    return 'Wyoming'
  else:
    return 'StateUnknown'

In [56]:
df['State'] = df.apply (zipcode_to_state, axis=1)

In [57]:
df.head(10)

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType,cleaned_zipcode,CarAgeTimeSale,State
State,,,,,,,,,,,,,,,,
Texas,137178,7500,2020,786**,84430,Ford,Mustang,1988,LX,5.0L Gas V8,Sedan,0,RWD,786.0,32,Texas
Colorado,96705,15000,2019,81006,0,Replica/Kit Makes,Jaguar Beck Lister,1958,NaN,383 Fuel injected,Convertible,8,RWD,810.0,61,Colorado
Florida,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD,334.0,25,Florida
New Jersey,80773,11600,2019,07852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD,78.0,51,New Jersey
New Jersey,64287,44000,2019,07728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD,77.0,17,New Jersey
Indiana,132695,950,2020,462**,71300,Mercury,Montclair,1965,NaN,NO ENGINE,Sedan,0,RWD,462.0,55,Indiana
New York,132829,950,2020,105**,71300,Mercury,Montclair,1965,NaN,NaN,Sedan,0,NaN,105.0,55,New York
New Jersey,5250,70000,2019,07627,6500,Land Rover,Defender,1997,NaN,4.0 Liter Fuel Injected V8,NaN,0,4WD,76.0,22,New Jersey
New Jersey,29023,1330,2019,07043,167000,Honda,Civic,2001,EX,NaN,Coupe,4,FWD,70.0,18,New Jersey


we need to check that no zipcode was missed and count the states that came back as 'StateUnknown'

In [58]:
df['State'].value_counts()

California        16968
Florida           12441
Pennsylvania       8841
Texas              6986
New York           6949
Michigan           6679
New Jersey         5115
Ohio               4546
Illinois           3621
Massachusetts      2988
Georgia            2861
Maryland           2826
Washington         2816
North Carolina     2604
Indiana            2510
Virginia           2297
Arizona            2242
Tennessee          1984
Colorado           1871
Connecticut        1808
Oregon             1756
Missouri           1613
Nevada             1599
Kentucky           1407
Wisconsin          1388
South Carolina     1380
Minnesota          1128
Alabamba           1052
Iowa                914
Idaho               847
Kansas              834
Oklahoma            833
Utah                759
Mississippi         746
Nebraska            745
Arkansas            607
Louisiana           592
New Mexico          536
West Virginia       512
Montana             441
Delaware            395
Washington DC   

we have qty * 3 states that show as state unknown. Print these for further investigation.

In [59]:
pd.set_option('display.max_columns', None)
print(df.loc[df['State'] == 'StateUnknown'])

Empty DataFrame
Columns: [ID, pricesold, yearsold, zipcode, Mileage, Make, Model, Year, Trim, Engine, BodyType, NumCylinders, DriveType, cleaned_zipcode, CarAgeTimeSale, State]
Index: []


there were 3 zipcodes that showed State unknown. Zipcode 1 is for destinations outside of USA. Zipcode 886 is not in use . Zipcode 963 is for Military bases in Japan. These 3 entries should be removed from out DataFrame.

In [60]:
df.drop(df.loc[df['State']== 'StateUnknown'].index, inplace=True)
pd.set_option('display.max_columns', None)
print(df.loc[df['State'] == 'StateUnkown'])

Empty DataFrame
Columns: [ID, pricesold, yearsold, zipcode, Mileage, Make, Model, Year, Trim, Engine, BodyType, NumCylinders, DriveType, cleaned_zipcode, CarAgeTimeSale, State]
Index: []


### reset the index 

reset the index on this Dataframe to be the State that the car was sold in.

In [ ]:
df=df.set_index('State')
df.head(20)

### join our dataframes

we now need to join our dataframes together to have our final dataframe that we will analysis for our project

### Final cleanup of Dataframe

In [ ]:
# drop the old zipcode coloumn
df.drop('zipcode', inplace=True, axis=1)
df.head(10)

In [ ]:
df= df.drop(["Trim"], axis=1)
df= df.drop(["Engine"], axis=1)
df= df.drop(["DriveType"], axis=1)
df= df.drop(["BodyType"], axis=1)

The entry of asterisks ( * ) in zipcode proved problematic for running of linear regression model.

In [ ]:
df.head(20)

our count for rows at this stage should be just over 122, 000 rows for the combined files in this Data Frame

In [ ]:
print(df)

            ID  pricesold  yearsold  ...               Model  Year NumCylinders
0       137178       7500      2020  ...             Mustang  1988            0
1        96705      15000      2019  ...  Jaguar Beck Lister  1958            8
2       119660       8750      2020  ...                 XJS  1995            6
3        80773      11600      2019  ...             Mustang  1968            8
4        64287      44000      2019  ...                 911  2002            6
...        ...        ...       ...  ...                 ...   ...          ...
122139   14948       4200      2019  ...             Mustang  1977            8
122140   58814       6500      2019  ...        E-Series Van  2012            0
122141    2156       2000      2019  ...              Bronco  1978            8
122142   29096       2280      2019  ...            3-Series  2000            6
122143   52391       5000      2019  ...               300ZX  1990            6

[122144 rows x 8 columns]


## <font color = green> This section will impute values for the missing information in the DataFrame


In [ ]:
# Count total NaN at each column in a DataFrame 
print(" \nCount total NaN at each column in a DataFrame : \n\n", 
      df.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 ID                0
pricesold         0
yearsold          0
Mileage           0
Make              0
Model           573
Year              0
NumCylinders      0
dtype: int64


In [ ]:
df.head(5)

,ID,pricesold,yearsold,Mileage,Make,Model,Year,NumCylinders
0,137178,7500,2020,84430,Ford,Mustang,1988,0
1,96705,15000,2019,0,Replica/Kit Makes,Jaguar Beck Lister,1958,8
2,119660,8750,2020,55000,Jaguar,XJS,1995,6
3,80773,11600,2019,97200,Ford,Mustang,1968,8
4,64287,44000,2019,40703,Porsche,911,2002,6


### <b> <font color = red> We need a section of code here to impute some of these missing values.

<b> <font color = red>code needs to be created here for functions that will impute whatever values we wish to impute. This will have to be done on a per location basis......
Also this has to be done for all values that we hope to impute.




## <font color = green> Cleaning up of Dataframe before proceeding

<b> <font color = red> we will remove rows with missing values during our testing. This ideally should not need to be done in the final setup as we will have imputed values and there should no missing rows in the DataFrame

In [ ]:
# remove rows with missing values
df.dropna(axis=0, how = 'any', thresh = None, subset = None, inplace= True)
# This should not have to be done in final version of this notebook

we can now replace text values with numeric values via one-hot encoding

In [ ]:
# convert non numeric entries using one hot encoding
df=pd.get_dummies(df, columns= ['Make','Model'])

In [ ]:
df.head(10)

,ID,pricesold,yearsold,Mileage,Year,NumCylinders,Make_1964 International,Make_300,Make_AC,Make_AM General,Make_AMC,Make_AMC / Rambler,Make_ASVE,Make_AUSTIN HEALEY,Make_Acura,Make_Aerodeck,Make_Alfa Romeo,Make_Alpine,Make_Amercan,Make_American Electric Vehicle,Make_American LaFrance,Make_American Motors,Make_Amphicar,Make_Ariel,Make_Arnolt-MG,Make_Aston Martin,Make_Auburn,Make_Audi,Make_Austin,Make_Austin Healey,Make_Austin-Healey,Make_Autobianchi,Make_Autocar,Make_Avanti,Make_BAW,Make_BISCUTER,Make_BMW,Make_BMW 3 Series,Make_BMW 335i,Make_BMW 530 Sport Pkg,...,Model_samurai,Model_savoy,Model_se,Model_sedan,Model_series 88 4x4 diesel especial station wagon,Model_side by side,Model_silverado 3500 HD LTZ,Model_solara,Model_speed 6,Model_spring,Model_sr5,Model_stag,Model_station wagon,Model_stinger,Model_street rod,Model_sunchaser,Model_sundance,Model_super beetle 2.0 turbo subaru wrx,Model_super blue 6,Model_super v63,Model_t bucket,Model_tC,Model_tasca,Model_three window coupe,Model_town & country,Model_truck,Model_truck 620,Model_tt quattro,Model_type 2,Model_type r,Model_unimog,Model_v-8,Model_vw bug,Model_x2,Model_x7,Model_xA,Model_xB,Model_xD,Model_xDrive28i,Model_yukon xl
0,137178,7500,2020,84430,1988,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,96705,15000,2019,0,1958,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,119660,8750,2020,55000,1995,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,80773,11600,2019,97200,1968,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,64287,44000,2019,40703,2002,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,132695,950,2020,71300,1965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,132829,950,2020,71300,1965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5250,70000,2019,6500,1997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,29023,1330,2019,167000,2001,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,80293,25200,2019,15000,1970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<b> <font color = red>We need to ensure that there are now no rows with missing values before we proceed.

Now do one final Sanity check that there are now missing values in any rows. Again this should not have to be done in the final setup.

In [ ]:
# Count total NaN at each column in a DataFrame 
print(" \nCount total NaN at each column in a DataFrame : \n\n", 
      df.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 ID                 0
pricesold          0
yearsold           0
Mileage            0
Year               0
                  ..
Model_xA           0
Model_xB           0
Model_xD           0
Model_xDrive28i    0
Model_yukon xl     0
Length: 4728, dtype: int64


## <font color = green> Logistic or linear regresssion can be ran here

<b> <font color = red> Here we need code to run the good old fashioned methods of whatever method we choose

Define the target(y) and non-target, learning variables (x)
We need to drop PricesOld, this is going to be Y, which we are hoping to predict

In [ ]:
x= df.drop('pricesold',1)
y=df['pricesold']


In [ ]:
# Splitting the Dataset to create test and train variable using the train 
# split command from sklearn
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, shuffle = True) 
# Test size is 25% of data . Train is 75%

print (x_train.shape, y_train.shape)    # This should have 75 percent of members
print (x_test.shape, y_test.shape)      # This should have 25 percent of members

(91178, 4727) (91178,)
(30393, 4727) (30393,)


### Firstly lets use the Linear regression model


In [ ]:
# Here we use the linear regression model. All columns must be numeric
# This numeric need was acheived by one-hot encoding above

lrm = linear_model.LinearRegression()


In [ ]:
lrm_model = lrm.fit(x_train, y_train)

In [ ]:
lrm_predictions= lrm.predict(x_test)

In [ ]:
print( ' Model Accuracy is :', lrm_model.score(x_test, y_test))

 Model Accuracy is : 0.3299379592364695


## <font color = green> This is Auto Machine learning section

<b> <font color = red> This is the first pass attempt at running machine learning on this setup
let the machine train itself

In [ ]:
# Assign X and Y variable
X = df.drop ('pricesold', axis = 1)
y = df['pricesold']

In [ ]:
# Split the Data in to test/train  set ( 70 / 30 split and shuffle)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3, shuffle = True)

In [ ]:
#Input algorrithim
model = ensemble.GradientBoostingRegressor()

In [ ]:
# Set the configurations that you wish to test

hyperparameters = {
    'n_estimators' : [200,300],
    'max_depth' : [4,6],
    'min_samples_split': [3,4],
    'min_samples_leaf': [5,6],
    'learning_rate': [0.01,0.02],
    'max_features': [0.8, 0.9],
    'loss': ['ls', 'lad', 'huber']
}

In [ ]:
# Define the grid search , run with 4 cpus in parallel
# It is worth setting verbose here or otherwise you get no output showing progress

grid_search= GridSearchCV(model, param_grid = hyperparameters, verbose = 1)

In [ ]:
# run the grid search on the training data
grid_search.fit(X_train, y_train)

NameError: ignored

In [ ]:
print (grid_search.best_params_)

NameError: ignored

In [ ]:
print("The best parameters are %s with a score of %0.2f" % (grid_search.best_params_, grid_search.best_score_))